<a href="https://colab.research.google.com/github/JaysonSuarez/IA/blob/main/ChatBot_Psicologo_RoBERTa_ClaseIA_Cohere.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ChatBot “Psicólogo”
Este cuaderno muestra paso a paso la construcción de un modelo de análisis de sentimientos con RoBERTa y la integración de un servicio de chat psicológico usando Cohere.

## Paso 1: Instalación de librerías

In [1]:
!pip install --upgrade transformers pandas scikit-learn cohere -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 24.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.0 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.
dask-cudf-cu12 25.2.2 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.0 which is incompatible.
cudf-cu12 25.2.1 requires pandas<2.2.4dev0,>=

Si se muestra un error en las dependencias es normal, no hay problema, igual ejecutará

## Paso 2: Carga manual del dataset InterTASS
Descomprimimos el ZIP subido a Colab, leemos todos los archivos TSV y concatenamos los datos de entrenamiento y validación.

- **zipfile**: módulo de Python para trabajar con archivos comprimidos.
- **glob**: encuentra rutas que coincidan con un patrón (aquí, todos los .tsv).
- **pandas**: para cargar y manipular los datos como DataFrames.

In [ ]:
import zipfile
import glob
import pandas as pd

# Ruta al ZIP subido
zip_path = 'Task1-train-dev.zip'
extract_dir = 'intertass'

# Descomprimir contenido
with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall(extract_dir)

def load_split(split_name):
    """
    Lee y concatena todos los archivos .tsv de un split ('train' o 'dev').
    Añade columna 'country' para distinguir el origen.
    """
    pattern = f"{extract_dir}/{split_name}/*.tsv"
    paths = glob.glob(pattern)
    dfs = []
    for path in paths:
        country = path.split('/')[-1].split('.')[0]
        df = pd.read_csv(path, sep='\t', header=None, names=['id','text','label'])
        df['country'] = country
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# Cargar splits
df_train = load_split('train')
df_dev   = load_split('dev')



# Mostrar tamaño de los conjuntos
print(f"Conjunto de entrenamiento: {df_train.shape[0]} muestras")
print(f"Conjunto de validación:    {df_dev.shape[0]} muestras")

Conjunto de entrenamiento: 4802 muestras
Conjunto de validación:    2443 muestras


## Paso 3: Exploración de datos
Examinamos algunas muestras y la distribución de etiquetas para comprender mejor la tarea:
- ¿Cuál es la proporción de clases?
- ¿Cómo es la longitud de los textos?
- ¿Qué tipo de emojis o abreviaturas aparecen?

In [ ]:
from IPython.display import display

# Mostrar primeras filas
display(df_train.head(10))

# Distribución de etiquetas
display(df_train['label'].value_counts())

,id,text,label,country
0,775087224857567232,Lo que mas amo de mi escritorio es que hay una...,P,pe
1,778485882647089152,Ese momento en el que no puedes dormir y no sa...,NEU,pe
2,774619666505400322,"@Fiorela_Gue aw ! recién veo esto, sorrry ! tu...",NEU,pe
3,802359837308887040,@CJSuasnabar soy el perro asesinado,N,pe
4,812398630137012224,@DavidJeffer06 Ay! Esas medias tan lindas,P,pe
5,816382826547269633,"Que histérico es la bestia, pobre bella",N,pe
6,813597935187304449,El quinto día #cousins #Holidays @ Previos Res...,NEU,pe
7,816876533231079424,Necesito urgente entrar a la gimnasia y al gim...,NEU,pe
8,818483269255761920,"Hasta lo peor de mi, quiere lo mejor para ti",NEU,pe
9,817071405040865281,"Mal momento para tomar café caliente, no dejo ...",N,pe


,count
label,
N,1885
NEU,1523
P,1394


## Paso 4: Preprocesamiento de texto y etiquetas
1. **Limpieza mínima**: eliminamos espacios extra y normalizamos mayúsculas.
2. **Codificación de etiquetas** con LabelEncoder para convertir las etiquetas de texto a enteros.
3. Mantenemos la columna 'country' para posibles análisis posteriores.

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Etiquetado numérico de las clases
le = LabelEncoder()
df_train['label_enc'] = le.fit_transform(df_train['label'])
df_dev['label_enc']   = le.transform(df_dev['label'])

# Mostrar mapeo de etiquetas
tag_map = dict(zip(le.classes_, le.transform(le.classes_)))
print('Mapeo de etiquetas:', tag_map)

Mapeo de etiquetas: {'N': np.int64(0), 'NEU': np.int64(1), 'P': np.int64(2)}


## Paso 5: Tokenización con RoBERTa
Convertimos textos a tensores que el modelo pueda procesar:
- **Padding** a longitud máxima para batches homogéneos.
- **Truncation** para no exceder la capacidad del modelo.
- **Max length**: elegimos 128 tokens como compromiso entre cobertura y velocidad.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('BSC-TeMU/roberta-base-bne')

def tokenize_df(df):
    return tokenizer(
        df['text'].tolist(),
        padding='max_length', truncation=True, max_length=128,
        return_tensors='pt'
    )

train_enc = tokenize_df(df_train)
dev_enc   = tokenize_df(df_dev)

# Verificar dimensiones
print('train input_ids shape:', train_enc['input_ids'].shape)
print('dev   input_ids shape:', dev_enc['input_ids'].shape)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.15M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.46M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

train input_ids shape: torch.Size([4802, 128])
dev   input_ids shape: torch.Size([2443, 128])


## Paso 6: Fine-Tuning del modelo RoBERTa
Entrenamos el modelo con los datos tokenizados, ajustando:
- **learning rate**: 2e-5 para ajustes suaves.
- **batch size**: 8 para balancear GPU y memoria.
- **epochs**: 3 ciclos completos para evitar sobreajuste.
- **report_to='none'**: desactiva logs externos (W&B).

In [ ]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import os
os.environ["WANDB_DISABLED"] = "true"

model = AutoModelForSequenceClassification.from_pretrained(
    'BSC-TeMU/roberta-base-bne', num_labels=len(le.classes_)
)
train_dataset = torch.utils.data.TensorDataset(
    train_enc['input_ids'], train_enc['attention_mask'], torch.tensor(df_train['label_enc'].values)
)
dev_dataset   = torch.utils.data.TensorDataset(
    dev_enc['input_ids'], dev_enc['attention_mask'],   torch.tensor(df_dev['label_enc'].values)
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    from sklearn.metrics import f1_score
    return {
        'accuracy': (preds == labels).mean(),
        'f1_macro': f1_score(labels, preds, average='macro')
    }

class IntertassDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = IntertassDataset(train_enc, df_train['label_enc'].values)
dev_dataset = IntertassDataset(dev_enc, df_dev['label_enc'].values)

training_args = TrainingArguments(
    output_dir='./results',
    do_train=True,
    do_eval=True,
    logging_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=2e-5,
    report_to='none'
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    compute_metrics=compute_metrics
)
trainer.train()

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at BSC-TeMU/roberta-base-bne and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-8-12ab29ef32e1>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,0.825700
1000,0.544900
1500,0.315300


<ipython-input-8-12ab29ef32e1>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-12ab29ef32e1>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-8-12ab29ef32e1>:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=1803, training_loss=0.5010116564453937, metrics={'train_runtime': 391.5136, 'train_samples_per_second': 36.796, 'train_steps_per_second': 4.605, 'total_flos': 947602973947392.0, 'train_loss': 0.5010116564453937, 'epoch': 3.0})

## Paso 7: Demostración interactiva y ChatBot Psicológico
Ingresa cualquier texto, primero clasificamos el sentimiento y luego enviamos el mensaje a Cohere para generar una respuesta empática.

Reemplaza `<<apiKey>>` con tu clave real de Cohere.

In [ ]:
import cohere
import torch

# Función para predecir sentimiento
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)

    # Move inputs to the same device as the model
    for key in inputs:
        inputs[key] = inputs[key].to(model.device)

    outputs = model(**inputs)
    pred = outputs.logits.argmax().item()
    return le.inverse_transform([pred])[0]

# Inicializar cliente de Cohere
co = cohere.ClientV2('uGJ50Z2ilig2S60S5GbsIcIEwp1FdzosdkXkDsm8')

print("""-Recuerda que este chatbot es solo una herramienta de apoyo y no un sustituto de la ayuda profesional.
Las respuestas que se generen son con fines de mejora y no deben ser consideradas como un diagnóstico o tratamiento profesional.-""")

while True:
    # Entrada del usuario
    text = input('Escribe tu mensaje para el ChatBot Psicólogo: ')

    # Verificar si el usuario quiere detener la conversación
    if text.lower() in ["stop", "basta", "alto", "fin", "terminar"]:
        break

    # Predecir el sentimiento del mensaje
    sentiment = predict_sentiment(text)
    print(f"Clasificación del mensaje: {sentiment}")

    # Llamada a Cohere Chat con contexto
    response = co.chat(
        model='command-a-03-2025',
        messages=[
            {'role': 'system', 'content': 'Eres un chat únicamente entrenado para dar respuestas de ayuda psicológica, ninguna otra. Solo debes dar respuestas mejorativas, y no debes dar diagnósticos o tratamientos, ya que no eres un profesional.'},
            {'role': 'user', 'content': text}
        ]
    )
    # response.message.content es una lista de objetos TextAssistantMessageResponseContentItem
    print(response.message.content[0].text)

Recuerda que este chatbot es solo una herramienta de apoyo y no un sustituto de la ayuda profesional. 
Las respuestas que se generen son con fines de mejora y no deben ser consideradas como un diagnóstico o tratamiento profesional.
